## Create Masks from Bounding Boxes

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import SimpleITK as sitk
from tqdm import tqdm

In [ ]:
def show_image(image, cmap="gray", ax=None):
    image_arr = sitk.GetArrayFromImage(image)
    image_arr = np.squeeze(image_arr)
    if ax is None:
        plt.imshow(image_arr, cmap=cmap)
    else:
        ax.imshow(image_arr, cmap=cmap)

In [ ]:
def remove_suffix(input_string, suffix):
    if suffix and input_string.endswith(suffix):
        return input_string[: -len(suffix)]
    return input_string

In [ ]:
def bounding_box_mask(image, label):
    """Generates a bounding box mask around a labeled region in an image

    Args:
        image (SimpleITK.Image): The input image.
        label (SimpleITK.Image): The labeled image containing the region of interest.

    Returns:
        SimpleITK.Image: An image containing the with the bounding box mask applied with the
        same spacing as the original image.

    Note:
        This function assumes that the input image and label are SimpleITK.Image objects.
        The returned bounding box mask is a binary image where pixels inside the bounding box
        are set to 1 and others are set to 0.
    """
    # get original spacing
    original_spacing = label.GetSpacing()

    # convert image and label to arrays
    image_array = sitk.GetArrayFromImage(image)
    image_array = np.squeeze(image_array)
    label_array = sitk.GetArrayFromImage(label)
    label_array = np.squeeze(label_array)

    # determine corners of the bounding box
    first_nonzero_row_index = np.nonzero(np.any(label_array != 0, axis=1))[0][0]
    last_nonzero_row_index = np.max(np.nonzero(np.any(label_array != 0, axis=1)))
    first_nonzero_column_index = np.nonzero(np.any(label_array != 0, axis=0))[0][0]
    last_nonzero_column_index = np.max(np.nonzero(np.any(label_array != 0, axis=0)))

    top_left_corner = (first_nonzero_row_index, first_nonzero_column_index)
    bottom_right_corner = (last_nonzero_row_index, last_nonzero_column_index)

    # define the bounding box as an array mask
    bounding_box_array = label_array.copy()
    bounding_box_array[
        top_left_corner[0] : bottom_right_corner[0] + 1,
        top_left_corner[1] : bottom_right_corner[1] + 1,
    ] = 1
    # add channel dimension
    bounding_box_array = bounding_box_array[None, ...]

    # get Image from Array Mask and apply original spacing
    bounding_box_image = sitk.GetImageFromArray(bounding_box_array)
    bounding_box_image.SetSpacing(original_spacing)
    return bounding_box_image

In [ ]:
def threshold_based_crop(image):
    """
    Use Otsu's threshold estimator to separate background and foreground. In medical imaging the background is
    usually air. Then crop the image using the foreground's axis aligned bounding box.
    Args:
        image (SimpleITK image): An image where the anatomy and background intensities form a
                                 bi-modal distribution
                                 (the assumption underlying Otsu's method.)
    Return:
        Cropped image based on foreground's axis aligned bounding box.
    """
    # Set pixels that are in [min_intensity,otsu_threshold] to inside_value, values above otsu_threshold are
    # set to outside_value. The anatomy has higher intensity values than the background, so it is outside.
    inside_value = 0
    outside_value = 255
    label_shape_filter = sitk.LabelShapeStatisticsImageFilter()
    label_shape_filter.Execute(sitk.OtsuThreshold(image, inside_value, outside_value))
    bounding_box = label_shape_filter.GetBoundingBox(outside_value)
    # The bounding box's first "dim" entries are the starting index and last "dim" entries the size
    return sitk.RegionOfInterest(
        image,
        bounding_box[int(len(bounding_box) / 2) :],
        bounding_box[0 : int(len(bounding_box) / 2)],
    )

In [ ]:
def mask_and_crop(image, label):
    """
    Performs masking and cropping operations on an image and its label.

    Args:
        image (SimpleITK.Image): The image to be processed.
        label (SimpleITK.Image): The corresponding label image.

    Returns:
        tuple: A tuple containing two SimpleITK.Image objects.
            - cropped_boxed_image: The image after applying bounding box masking and cropping.
            - mask: The binary mask corresponding to the label after cropping.

    Note:
        This function relies on other functions: bounding_box_mask() and threshold_based_crop().
    """
    box_mask = bounding_box_mask(image, label)
    boxed_image = sitk.Mask(image, box_mask, maskingValue=0, outsideValue=0)
    masked_image = sitk.Mask(image, label, maskingValue=0, outsideValue=0)

    cropped_boxed_image = threshold_based_crop(boxed_image)
    cropped_masked_image = threshold_based_crop(masked_image)

    mask = np.squeeze(sitk.GetArrayFromImage(cropped_masked_image))
    mask = np.where(mask > 0, 1, 0)
    mask = sitk.GetImageFromArray(mask[None, ...])
    return cropped_boxed_image, mask

## Box Masks Large Example usage Step by Step

In [ ]:
# read image and label from path
test_image = sitk.ReadImage("../test_data/pelvis.nii.gz")
test_label = sitk.ReadImage("../test_data/label.nii.gz")

# create bounding box mask
box_mask = bounding_box_mask(test_image, test_label)

# show original mask and box mask
fig, axs = plt.subplots(1, 2, figsize=(10, 20))

show_image(test_label, ax=axs[0])
show_image(box_mask, ax=axs[1])

In [ ]:
# apply masks to the images
masked_image = sitk.Mask(test_image, test_label, maskingValue=0, outsideValue=0)
boxed_image = sitk.Mask(test_image, box_mask, maskingValue=0, outsideValue=0)

fig, axs = plt.subplots(1, 2, figsize=(10, 20))

# show image with original mask and box mask applied
show_image(masked_image, ax=axs[0])
show_image(boxed_image, ax=axs[1])

In [ ]:
# crop the images to the bounding box
cropped_masked_image = threshold_based_crop(masked_image)
cropped_boxed_image = threshold_based_crop(boxed_image)

fig, axs = plt.subplots(1, 2, figsize=(10, 20))

# show images
show_image(cropped_masked_image, ax=axs[0])
show_image(cropped_boxed_image, ax=axs[1])

In [ ]:
# display the mask on top of the new image
finished_image, finished_label = mask_and_crop(test_image, test_label)

finished_label = np.squeeze(sitk.GetArrayFromImage(finished_label))
finished_image = np.squeeze(sitk.GetArrayFromImage(finished_image))

plt.imshow(finished_image, cmap="gray")
plt.imshow(finished_label, alpha=0.5, cmap="jet")

## Box Masks Large: Process the Dataset

In [ ]:
# define dataset
df = pd.read_csv("../data/dataset.csv")

df.head()
new_df = df

In [ ]:
# create directory to store the processed images, labels and csv
output_dir_im = "../data/boxed_grayscale_niftis/image"
output_dir_label = "../data/boxed_grayscale_niftis/label"

csv_output_path = "../data/boxed_grayscale_niftis/"

os.makedirs(output_dir_im, exist_ok=True)
os.makedirs(output_dir_label, exist_ok=True)

In [ ]:
pbar = tqdm(total=len(df))

for index, row in df.iterrows():
    image = sitk.ReadImage(row["image"])
    label = sitk.ReadImage(row["label"])

    file_name_im = remove_suffix(os.path.basename(row["image"]), ".nii.gz")
    file_name_label = remove_suffix(os.path.basename(row["label"]), ".nii.gz")

    destination_im = os.path.join(output_dir_im, file_name_im + ".nii.gz")
    destination_label = os.path.join(output_dir_label, file_name_label + ".nii.gz")

    finished_image, finished_label = mask_and_crop(image, label)

    new_df.loc[index, "image"] = destination_im
    new_df.loc[index, "label"] = destination_label

    sitk.WriteImage(finished_image, destination_im)
    sitk.WriteImage(finished_label, destination_label)

    pbar.update(1)

new_df.to_csv(os.path.join(csv_output_path, "dataset.csv"), index=False)
pbar.close()